<a href="https://colab.research.google.com/github/stepan-ha/Labs/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Алгоритм розрахунку індивідуальних вхідних даних

---

Встановіть значення змінної variant: сума номера групи помноженого на 25 і
порядкового номеру студента в списку групи (групі ПМОм-11 відповідає номер
0, групі ПМІм-11 відповідає номер 1, групі ПМІм-12 відповідає номер 2, групі
ПМІм-13 відповідає номер 3). Далі встановіть set.seed(variant) та згенеруйте
значення змінної redundant як заокруглене до цілого (для заокруглення можна
використати функції floor або round) випадкове число з рівномірного на інтервалі
(номер групи + 5, 25 – номер групи) розподілу (функція runif). Також згенеруйте
значення змінної year як заокруглене до цілого випадкове число з рівномірного
на інтервалі (2006, 2008) розподілу.


In [30]:
import pandas as pd
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


Розрахунок індивідуальних даних
---



In [64]:
group_number = 2

student_number = 5

variant = group_number * 25 + student_number
print(f"Мій номер індивідуального завдання: {variant}")

np.random.seed(variant)

redundant = math.floor(random.uniform(group_number + 5, 25 - group_number))
print(f"Redundant: {redundant}%")

year = np.random.randint(2006, 2008)

print(year)

Мій номер індивідуального завдання: 55
Redundant: 10%
2007


# Допоміжні функції

---


In [65]:
def checkKNAccuracy(x_train, y_train, x_test, y_test):
  best_accuracy = 0
  best_k = 1
  for k in range(1, 21):
      knn = KNeighborsClassifier(n_neighbors=k)
      knn.fit(x_train, y_train)
      accuracy = accuracy_score(y_test, knn.predict(X_test))
      if accuracy > best_accuracy:
          best_accuracy = accuracy
          best_k = k
  print(f"Найкраще K = {best_k}  з точністю: {best_accuracy}!")


def buildConfusionMatrix(y_test, prediction, methodName):
  print(f"\nМатриця помилок для {methodName}:")
  print(confusion_matrix(y_test, prediction))


# Завдання 1

---

Використовуючи дані Weekly, встановивши seed, що дорівнює значенню
змінної variant, побудуйте модель логістичної регресії для передбачення
Direction з використанням навчальних даних з 1990 по year рр., на основі єдиного
предиктора Lag2. Обчисліть матрицю помилок та загальну частку правильних
прогнозів на тестових даних (тобто даних за year-2010 роки). За аналогічних
умов використайте для передбачення Direction лінійний дискримінантний аналіз,
квадратичний дискримінантний аналіз та метод K-найближчих сусідів з K = 1.
Порівняйте використані методи. За якого K точність методу K-найближчих
сусідів буде найбільшою?

In [66]:

data = pd.read_csv('Weekly.csv')

train_data = data[(data['Year'] >= 1990) & (data['Year'] < year)]
test_data = data[(data['Year'] >= year) & (data['Year'] <= 2010)]

X_train = train_data[['Lag2']]
y_train = train_data['Direction']
X_test = test_data[['Lag2']]
y_test = test_data['Direction']

log_model = LogisticRegression()
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
y_pred_lda = lda_model.predict(X_test)

qda_model = QuadraticDiscriminantAnalysis()
qda_model.fit(X_train, y_train)
y_pred_qda = qda_model.predict(X_test)

knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)
y_pred_knn1 = knn_model.predict(X_test)

print("Точність моделі логістичної регресії:", accuracy_score(y_test, y_pred_log))
buildConfusionMatrix(y_test, y_pred_log, "Моделі логістичної регресії");

print("Точність лінійного дискримінантного аналізу:", accuracy_score(y_test, y_pred_lda))
buildConfusionMatrix(y_test, y_pred_lda, "Лінійного дискримінантного аналізу");

print("Точність квадратичного дискримінантного аналізу:", accuracy_score(y_test, y_pred_qda))
buildConfusionMatrix(y_test, y_pred_qda, "Квадратичного дискримінантного аналізу");

print("Точність методу K-найближчих сусідів (K=1):", accuracy_score(y_test, y_pred_knn1))
buildConfusionMatrix(y_test, y_pred_knn1, "Методу K-найближчих сусідів");
checkKNAccuracy(X_train, y_train, X_test, y_test);



Точність моделі логістичної регресії: 0.5502392344497608

Матриця помилок для Моделі логістичної регресії:
[[  5  91]
 [  3 110]]
Точність лінійного дискримінантного аналізу: 0.5502392344497608

Матриця помилок для Лінійного дискримінантного аналізу:
[[  5  91]
 [  3 110]]
Точність квадратичного дискримінантного аналізу: 0.5406698564593302

Матриця помилок для Квадратичного дискримінантного аналізу:
[[  0  96]
 [  0 113]]
Точність методу K-найближчих сусідів (K=1): 0.5119617224880383

Матриця помилок для Методу K-найближчих сусідів:
[[49 47]
 [55 58]]
Найкраще K = 19  з точністю: 0.5980861244019139!


# Вигляд матриці для аналізу

---
[

  
  [True Negative, False Positive]

  [False Negative, True Positive]

]

# Завдання 2

---

Модифікуйте дані Auto наступним чином: встановивши seed, що дорівнює
значенню змінної variant, видаліть redundant % спостережень з допомогою
функції sample. Створіть двійкову змінну mpg01, яка містить 1, якщо mpg містить
значення більше за середнє, і 0, якщо mpg містить значення менше за середнє.
Розбийте дані на навчальний та тестовий набори. При розбитті набору даних
обов’язково!!! встановити seed, що дорівнює значенню змінної variant, та
використати функцію sample. Обсяг тестової вибірки виберіть 2*redundant % від
загального обсягу даних. Застосуйте лінійний дискримінантний аналіз,
квадратичний дискримінантний аналіз, логістичну регресію та метод Кнайближчих сусідів з різними значеннями для К на навчальних даних, щоб
передбачити mpg01 на основі змінних weight, displacement та horsepower.
Порівняйте тестові помилка використаних моделей. За якого K точність методу
K-найближчих сусідів буде найбільшою та якою буде точність цього методу?

In [67]:

data_auto = pd.read_csv('Auto.csv')

print(data_auto.head())

Auto_new = data_auto.drop(data_auto.sample(frac=redundant / 100, random_state=variant).index)

print(f"Початковий розмір: {data_auto.shape}")
print(f"Новий розмір: {Auto_new.shape}")

invalid_rows = Auto_new[Auto_new['horsepower'].apply(lambda x: isinstance(x, str) and x.strip() == '?')]

Auto_new['horsepower'] = pd.to_numeric(Auto_new['horsepower'], errors='coerce')

Auto_new = Auto_new.dropna(subset=['horsepower'])

#цю валідацію, а також конвертацію я зробив через помилки,
#які виникали під час лінійної регресії, а також щоб результат був максимально точним

Auto_new['mpg01'] = np.where(Auto_new['mpg'] > Auto_new['mpg'].mean(), 1, 0)

X = Auto_new[['weight', 'displacement', 'horsepower']]
y = Auto_new['mpg01']

sizeP = ((2*redundant)/100)

print(f"sizeP: {sizeP}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = sizeP, random_state=variant)

log_model_auto = LogisticRegression()
log_model_auto.fit(X_train, y_train)
y_pred_log_auto = log_model_auto.predict(X_test)

lda_model_auto = LinearDiscriminantAnalysis()
lda_model_auto.fit(X_train, y_train)
y_pred_lda_auto = lda_model_auto.predict(X_test)

qda_model_auto = QuadraticDiscriminantAnalysis()
qda_model_auto.fit(X_train, y_train)
y_pred_qda_auto = qda_model_auto.predict(X_test)

knn_model_auto = KNeighborsClassifier(n_neighbors=1)
knn_model_auto.fit(X_train, y_train)
y_pred_knn1_auto = knn_model_auto.predict(X_test)

print("Точність моделі логістичної регресії:", accuracy_score(y_test, y_pred_log_auto))
buildConfusionMatrix(y_test, y_pred_log_auto, "Моделі логістичної регресії")

print("Точність лінійного дискримінантного аналізу:", accuracy_score(y_test, y_pred_lda_auto))
buildConfusionMatrix(y_test, y_pred_lda_auto, "Лінійного дискримінантного аналізу")

print("Точність квадратичного дискримінантного аналізу:", accuracy_score(y_test, y_pred_qda_auto))
buildConfusionMatrix(y_test, y_pred_qda_auto, "Квадратичного дискримінантного аналізу")

print("Точність методу K-найближчих сусідів (K=1):", accuracy_score(y_test, y_pred_knn1_auto))
buildConfusionMatrix(y_test, y_pred_knn1_auto, "Методу K-найближчих сусідів")

checkKNAccuracy(X_train, y_train, X_test, y_test)


    mpg  cylinders  displacement horsepower  weight  acceleration  year  \
0  18.0          8         307.0        130    3504          12.0    70   
1  15.0          8         350.0        165    3693          11.5    70   
2  18.0          8         318.0        150    3436          11.0    70   
3  16.0          8         304.0        150    3433          12.0    70   
4  17.0          8         302.0        140    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  
Початковий розмір: (397, 9)
Новий розмір: (357, 9)
sizeP: 0.2
Точність моделі логістичної регресії: 0.8591549295774648

Матриця помилок для Моделі логістичної регресії:
[[33  8]
 [ 2 28]]
Точність лінійного дискримінантного аналізу: 0.8169014084507042

Матриця помилок для Лінійного дискримінантного аналізу:
[[30 1